In [ ]:
import pandas as pd
import os
import matplotlib.pylab as plt
import xarray as xr
import numpy as np
from scipy.signal import butter, filtfilt, freqz
from matplotlib.ticker import ScalarFormatter
from matplotlib.dates import DateFormatter
from glob import glob
import gsw
import cartopy.crs as ccrs
from matplotlib.colors import LightSource
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from matplotlib.colors import LightSource, LinearSegmentedColormap, Normalize
from matplotlib.cm import ScalarMappable
import matplotlib.gridspec as gridspec
from matplotlib.gridspec import GridSpec
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.dates as mdates
from math import radians, cos, sin, sqrt, atan2
from itertools import combinations 
import h5py
import requests
from scipy.linalg import lstsq

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import numpy as np
from matplotlib.colors import Normalize
from matplotlib.colors import LightSource

from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.patheffects as pe

In [ ]:
def add_scalebar(ax, center_lon, center_lat, length_km, label=None, fontsize=10, color='black'):
    length_deg = length_km / degree_to_km(center_lat)
    scalebar_x = [center_lon, center_lon + length_deg]
    scalebar_y = [center_lat, center_lat]
    ax.plot(scalebar_x, scalebar_y, transform=ccrs.PlateCarree(), color=color, linewidth=4, 
            path_effects=[pe.withStroke(linewidth=8, foreground="white")],zorder=10)
    if label is None:
        label = f'{length_km} km'
    ax.text(center_lon + length_deg / 2, scalebar_y[0] + (0.07 * length_deg), label, transform=ccrs.PlateCarree(),
            horizontalalignment='center', verticalalignment='bottom', fontsize=fontsize, color=color,
            path_effects=[pe.withStroke(linewidth=3, foreground="white")])
def degree_to_km(lat):
    radius_of_earth = 6371.01
    return np.pi / 180 * radius_of_earth * np.cos(np.deg2rad(lat))

In [ ]:
big_map_bathy_file = r"H:\Chapter3\LCH\MGDS_Download\SRTM15_V2.7.nc"
axial_bathy_file = r"H:\Chapter3\Axial\bathy\GMRTv4_2_20240530topo.xyz"
endeavour_bathy_file = r"H:\Chapter3\Endeavor\bathy\Endeav30_UW.xyz"
ncfile = r"H:\Chapter3\LCH\MGDS_Download\SRTM15v2_1m.nc"

## OOI and ONC cable map

In [ ]:
import pandas as pd
import geojson
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# --- Load OOI cable route ---
ooi_df = pd.read_csv("ooi_cables.csv")
latt1 = ooi_df['Lat'].tolist()
lonn1 = ooi_df['Long'].tolist()

# --- Load ONC cable route ---
with open('onc_cables.geoJson') as f:
    gj = geojson.load(f)
onc_coords = []
for feature in gj['features']:
    onc_coords.append(feature.geometry.coordinates[0])

# --- Define nodes ---
ooi_py = [44.48146, 44.36364, 44.69139, 45.755555]
ooi_px = [-125.14833, -124.96199, -124.45702, -127.278438]

ins_y = [44.5674, 44.5096]
ins_x = [-125.1442, -125.3983]

onc_py = [48.34590333, 47.95836833, 48.67453667, 47.74129884, 48.81321833]
onc_px = [-126.1580133, -129.0354383, -126.85251, -127.7294497, -125.2810783]

GNSS_x = [-126.5, -127.1]   # Dummy GNSS locations – replace with your real values
GNSS_y = [48.0, 48.2]
GNSS_xx = [-126.8]
GNSS_yy = [47.9]

# --- Create map ---
fig = plt.figure(figsize=(8, 10))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
ax.set_extent([-130, -123, 44, 49], crs=ccrs.PlateCarree())

# --- Add coastlines and land features ---
ax.add_feature(cfeature.LAND, facecolor='lightgray')
ax.add_feature(cfeature.COASTLINE)
ax.gridlines(draw_labels=True)

# --- Plot OOI cable in two segments ---
ax.plot(lonn1[0:333], latt1[0:333], color='black', linewidth=1.3, transform=ccrs.PlateCarree())
ax.plot(lonn1[333:500], latt1[333:500], color='black', linewidth=1.3, transform=ccrs.PlateCarree())

# --- Plot OOI nodes ---
for i in range(3):
    ax.plot(ooi_px[i], ooi_py[i], marker='s', color='red', markersize=6, transform=ccrs.PlateCarree())
ax.plot(ooi_px[3], ooi_py[3], marker='s', color='black', markersize=6, transform=ccrs.PlateCarree())

# --- Plot INS stations (triangles and squares) ---
#ax.plot(ins_x[0], ins_y[0], marker='^', color='gold', markersize=6, transform=ccrs.PlateCarree())
ax.plot(ins_x[1], ins_y[1], marker='s', color='red', markersize=6, transform=ccrs.PlateCarree())

# --- Plot ONC cable lines ---
for coord_list in onc_coords:
    clons = [c[0] for c in coord_list]
    clats = [c[1] for c in coord_list]
    ax.plot(clons, clats, color='black', linewidth=1.5, transform=ccrs.PlateCarree())

# --- Plot ONC nodes ---
ax.plot(onc_px, onc_py, marker='s', color='black', linestyle='None', markersize=8, transform=ccrs.PlateCarree())

# --- Plot GNSS-A sites ---
#ax.plot(GNSS_x, GNSS_y, marker='^', color='white', markersize=8, transform=ccrs.PlateCarree())
#ax.plot(GNSS_xx, GNSS_yy, marker='^', color='white', markersize=8, transform=ccrs.PlateCarree())

# --- Final marker for endpoint? ---
#ax.plot(lonn1[332], latt1[332], marker='^', color='red', markersize=6, transform=ccrs.PlateCarree())

# --- Title and layout ---
ax.set_title("OOI and ONC Cable Layout with Stations", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
cmap = plt.cm.YlGnBu_r


import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import numpy as np
from matplotlib.colors import Normalize
from matplotlib.colors import LightSource

ncfile = big_map_bathy_file
data = xr.open_dataset(ncfile)

region = [-132, -123, 41.5, 52.5]
subset = data.sel(lon=slice(region[0], region[1]), lat=slice(region[2], region[3]))

bathy_x = subset.lon.values
bathy_y = subset.lat.values
bathy_z = subset.z.values


bathy_z[bathy_z > 0] = np.nan

bathy_z_flipped = np.flipud(bathy_z)

ls = LightSource(azdeg=300, altdeg=30)
hillshade = ls.hillshade(bathy_z_flipped, vert_exag=1)
valid_bathy_z = bathy_z_flipped[~np.isnan(bathy_z_flipped)]
norm = Normalize(vmin=np.nanmin(valid_bathy_z)+700, vmax=np.nanmax(valid_bathy_z)-200)

masked_bathy_z = np.ma.array(bathy_z_flipped, mask=np.isnan(bathy_z_flipped))
rgba = cmap(norm(masked_bathy_z))
shaded = ls.shade(masked_bathy_z, cmap=cmap, norm=norm, blend_mode='overlay', vert_exag=0.5, dx=1, dy=1, fraction=0.7)

fig = plt.figure(figsize=(6, 9))
ax1 = fig.add_subplot(1, 1, 1, projection=ccrs.PlateCarree())
# --- Add coastlines and land features ---
ax1.add_feature(cfeature.LAND, facecolor='lightgray')
ax1.add_feature(cfeature.COASTLINE)

# --- Plot OOI cable in two segments ---
#ax1.plot(lonn1[0:333], latt1[0:333], color='black', linewidth=1.3, transform=ccrs.PlateCarree())
#ax1.plot(lonn1[333:500], latt1[333:500], color='black', linewidth=1.3, transform=ccrs.PlateCarree())
#
## --- Plot OOI nodes ---
#for i in range(3):
#    ax1.plot(ooi_px[i], ooi_py[i], marker='s', color='gold', markersize=4, transform=ccrs.PlateCarree())
#ax1.plot(ooi_px[3], ooi_py[3], marker='s', color='gold', markersize=4, transform=ccrs.PlateCarree())
#
## --- Plot INS stations (triangles and squares) ---
##ax.plot(ins_x[0], ins_y[0], marker='^', color='gold', markersize=6, transform=ccrs.PlateCarree())
#ax1.plot(ins_x[1], ins_y[1], marker='s', color='gold', markersize=4, transform=ccrs.PlateCarree())

# --- Plot ONC cable lines ---
for coord_list in onc_coords:
    clons = [c[0] for c in coord_list]
    clats = [c[1] for c in coord_list]
    ax1.plot(clons, clats, color='black', linewidth=1.5, transform=ccrs.PlateCarree())

# --- Plot ONC nodes ---
ax1.plot(onc_px, onc_py, marker='s', color='red', linestyle='None', markersize=4, transform=ccrs.PlateCarree())

# New node coordinates
node_lats = [45.8205, 45.9413]
node_lons = [-129.7432, -129.9697]

# Plot red squares for the new nodes
#ax1.plot(node_lons, node_lats, 
#         marker='s', markersize=6, 
#         color='gold', linestyle='None', 
#         transform=ccrs.PlateCarree(), 
#         label='New Nodes')

#ax1.add_feature(cfeature.LAND, zorder=0, facecolor='dimgray')
#ax1.add_feature(cfeature.OCEAN, zorder=0)
#
#ax1.add_feature(cfeature.COASTLINE, edgecolor='white', zorder=1)
#ax1.add_feature(cfeature.BORDERS, linestyle=':', edgecolor='white', zorder=1)
#states_provinces = cfeature.NaturalEarthFeature(
#    category='cultural',
#    name='admin_1_states_provinces_lines',
#    scale='50m',
#    facecolor='none')
#ax1.add_feature(states_provinces, edgecolor='white', zorder=1)

ax1.imshow(shaded, extent=(region[0], region[1], region[2], region[3]), transform=ccrs.PlateCarree(), zorder=1)

sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])
cbar = plt.colorbar(sm, ax=ax1, orientation='horizontal', pad=0.05)
cbar.set_label('Depth (m)')

gl = ax1.gridlines(draw_labels=True, alpha=0.000001)
gl.top_labels = False
gl.right_labels = False

add_scalebar(ax1, center_lon=-126, center_lat=42.5, length_km=200)

xlim1 = [-129.16, -129]
ylim1 = [47.89, 48]
ax1.plot([xlim1[0], xlim1[1], xlim1[1], xlim1[0], xlim1[0]],
         [ylim1[0], ylim1[0], ylim1[1], ylim1[1], ylim1[0]],
         color='purple', transform=ccrs.PlateCarree(), linewidth=2)

#xlim2 = [-130.12, -129.68]
#ylim2 = [45.75, 46.04]
#ax1.plot([xlim2[0], xlim2[1], xlim2[1], xlim2[0], xlim2[0]],
#         [ylim2[0], ylim2[0], ylim2[1], ylim2[1], ylim2[0]],
#         color='purple', transform=ccrs.PlateCarree(), linewidth=2)

#xlim3 = [-126, -124]
#ylim3 = [44, 45]
#ax1.plot([xlim3[0], xlim3[1], xlim3[1], xlim3[0], xlim3[0]],
#         [ylim3[0], ylim3[0], ylim3[1], ylim3[1], ylim3[0]],
#         color='purple', transform=ccrs.PlateCarree(), linewidth=2);

# === New dashed-line rectangle ===
#xlim_dash = [-130, -122]
#ylim_dash = [42, 52]
#ax1.plot([xlim_dash[0], xlim_dash[1], xlim_dash[1], xlim_dash[0], xlim_dash[0]],
#        [ylim_dash[0], ylim_dash[0], ylim_dash[1], ylim_dash[1], ylim_dash[0]],
#        color='white', linestyle='--', linewidth=2, transform=ccrs.PlateCarree())
#plt.savefig('Ch3_big_map.png', dpi=300, bbox_inches='tight', transparent=True)

## Axial Seamount Map

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patheffects as pe
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from cartopy.mpl.ticker import LongitudeFormatter
from matplotlib.colors import LinearSegmentedColormap, Normalize, LightSource

# -------------------------
# Figure & layout
# -------------------------
FIG_W, FIG_H = 6, 7
fig = plt.figure(figsize=(FIG_W, FIG_H), dpi=160)
gs  = gridspec.GridSpec(
    2, 1, height_ratios=[2, 1],
    left=0.10, right=0.98, top=0.97, bottom=0.10, hspace=0.18, figure=fig
)

# -------------------------
# Map (top) — keep your exact window
# -------------------------
xlim0 = (-130.05, -129.96)
ylim0 = ( 45.90,   46.00)

ax_map = fig.add_subplot(gs[0], projection=ccrs.PlateCarree())

# color style + hillshade (unchanged)
vmin, vmax = -1700.0, -1400.0
norm = Normalize(vmin=vmin, vmax=vmax)
cmap = LinearSegmentedColormap.from_list("axial_ref", ["#e9d8a6", "#bfe3e3", "#31688e"], N=256)
ls   = LightSource(azdeg=300, altdeg=25)

Zm  = np.ma.masked_invalid(bathy_z)
rgb = ls.shade(Zm, cmap=cmap, norm=norm, blend_mode="overlay", vert_exag=2.0, fraction=0.70)

ax_map.imshow(
    rgb,
    extent=(np.nanmin(bathy_x), np.nanmax(bathy_x),
            np.nanmin(bathy_y), np.nanmax(bathy_y)),
    origin='upper', transform=ccrs.PlateCarree(),
    interpolation="bilinear", zorder=1
)
ax_map.set_xlim(xlim0); ax_map.set_ylim(ylim0)

# subtle contours + rim
ax_map.contour(bathy_x, bathy_y, bathy_z, levels=np.arange(vmin, vmax, 25),
               colors='k', linewidths=0.5, alpha=0.6, transform=ccrs.PlateCarree(), zorder=2)
rim = np.nanpercentile(bathy_z, 80)
ax_map.contour(bathy_x, bathy_y, bathy_z, levels=[rim],
               colors='white', linewidths=2.0, transform=ccrs.PlateCarree(), zorder=3)

# sites (C0–C3)
sites = [
    ("ASHES\nMJ03B",           -130.01386, 45.93359),
    ("Central Caldera\nMJ03F", -130.00877, 45.95485),
    ("International District\nMJ03D", -129.97750, 45.92579),
    ("Eastern Caldera\nMJ03E", -129.97411, 45.93989),
]
site_colors = ["C0", "C1", "C2", "C3"]
DOT_SIZE, LAB_FS = 80, 11
for (text, lon, lat), col in zip(sites, site_colors):
    ax_map.scatter(lon, lat, s=DOT_SIZE, c=col, edgecolor="black", lw=0.8,
                   transform=ccrs.PlateCarree(), zorder=4)
    ax_map.text(lon + 0.005, lat + 0.004, text, transform=ccrs.PlateCarree(),
                ha="center", va="bottom", color=col, fontsize=LAB_FS, weight="bold",
                path_effects=[pe.withStroke(linewidth=1.2, foreground="black")],
                bbox=dict(facecolor="white", edgecolor="none", alpha=0.6,
                          boxstyle="round,pad=0.12"),
                zorder=5)

# gridlines + title
gl = ax_map.gridlines(draw_labels=True, linestyle='--', color='gray', alpha=0.5)
gl.top_labels = False; gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER; gl.yformatter = LATITUDE_FORMATTER

ax_map.text(-129.99, 45.993, 'Axial Seamount', transform=ccrs.PlateCarree(),
            ha='center', color='black', size=20,
            bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, boxstyle='round,pad=0.1'))

# optional: profile line drawn on the map (df_profile assumed loaded)
ax_map.plot(df_profile['Longitude'], df_profile['Latitude'],
            transform=ccrs.PlateCarree(), linestyle='--',
            color='black', linewidth=2, zorder=1)

ax_map.patch.set_edgecolor('black')
ax_map.patch.set_linewidth(1)
add_scalebar(ax_map, center_lon=-129.98, center_lat=45.91, length_km=2)

# -------------------------
# Profile (bottom) — match width to the map (no map cropping)
# -------------------------
ax2 = fig.add_subplot(gs[1])

# 1) Use the same longitude limits as the map
ax2.set_xlim(xlim0)
ax2.xaxis.set_major_formatter(LongitudeFormatter())

# 2) After axes exist, set the bottom axis POSITION to use the SAME LEFT EDGE
#    and the SAME WIDTH as the map axis (this is what visually aligns widths)
map_box = ax_map.get_position()           # Bbox in figure fraction
bottom_h = 0.24                           # height of profile panel (tweak if you like)
gap      = 0.06                           # vertical gap between panels
bottom_y = map_box.y0 - gap - bottom_h    # place below the map
ax2.set_position([map_box.x0, bottom_y, map_box.width, bottom_h])

# ------- build the profile from df_profile, CLIPPED to map longitudes -------
lon_series  = pd.to_numeric(df_profile['Longitude'], errors='coerce')
elev_series = pd.to_numeric(df_profile['Elevation'], errors='coerce')  # meters
prof = pd.DataFrame({'lon': lon_series, 'elev_m': elev_series}).dropna().sort_values('lon')
mask = (prof['lon'] >= xlim0[0]) & (prof['lon'] <= xlim0[1])
prof_clip = prof.loc[mask] if mask.any() else prof

lons  = prof_clip['lon'].values
elevs = prof_clip['elev_m'].values

# main profile (+ optional smooth)
ax2.plot(lons, elevs/1000.0, color='black', lw=1.6)
# smooth = pd.Series(elevs).rolling(window=21, center=True, min_periods=3).mean().to_numpy()
# ax2.plot(lons, smooth/1000.0, color='black', lw=1.2, ls='--')

ax2.set_ylabel('Depth (km bsf)')
ax2.grid(True, color='0.85', lw=1)

# steepest slope (on clipped data)
mean_lat = np.nanmean(pd.to_numeric(df_profile['Latitude'], errors='coerce').values)
deg2m = 111e3 * np.cos(np.radians(mean_lat))
if len(lons) > 1:
    dxm = np.diff(lons) * deg2m
    dzm = np.diff(elevs)
    local_slopes = dzm / dxm
    i = np.argmax(np.abs(local_slopes))
    mid_lon  = (lons[i] + lons[i+1]) / 2
    mid_elev = (elevs[i] + elevs[i+1]) / 2
    steep = abs(np.degrees(np.arctan(local_slopes[i])))
    ax2.plot(mid_lon, mid_elev/1000.0, 'rv', markersize=8)
    ax2.text(0.40, 0.93, f"Steepest gradient: {steep:.2f}°",
             transform=ax2.transAxes, fontsize=12,
             bbox=dict(facecolor='white', alpha=0.85, boxstyle='round,pad=0.2'))

# optional slope segments (auto-clip to xlim)
lon1_a, lat1_a, elev1_a = -130.0310, 45.9501, -1427.1
lon1_b, lat1_b, elev1_b = -130.2160, 45.9497, -2017.4
dx1 = (lon1_b - lon1_a) * deg2m; dz1 = elev1_b - elev1_a
slope1_deg = np.degrees(np.arctan(dz1 / dx1))
ax2.plot([lon1_a, lon1_b], [elev1_a/1000, elev1_b/1000], 'k--')
ax2.text(-130.00, -1.60, f"gradient: {slope1_deg:.2f}°", fontsize=12, ha='center')

lon2_a, lat2_a, elev2_a = -129.5406, 45.9497, -2643.0
lon2_b, lat2_b, elev2_b = -130.0310, 45.9501, -1427.1
dx2 = abs((lon2_b - lon2_a) * deg2m); dz2 = abs(elev2_b - elev2_a)
slope2_deg = np.degrees(np.arctan(dz2 / dx2))
ax2.plot([lon2_a, lon2_b], [elev2_a/1000, elev2_b/1000], 'k--')
ax2.text(-130.035, -1.40, f"gradient: {slope2_deg:.2f}°", fontsize=12, ha='center')

# tidy y-range
y = elevs/1000.0
if np.isfinite(y).any():
    ax2.set_ylim(np.nanmin(y)-0.3, np.nanmax(y)+0.15)

plt.show()


## Endeavour Segment Map

In [ ]:
fig = plt.figure(figsize=(6, 8))
gs = GridSpec(1, 1, figure=fig)

ax1 = fig.add_subplot(111, projection=ccrs.PlateCarree())

xlim = [-129.16, -128.985]
ylim = [47.89, 48.03]

ls = LightSource(azdeg=300, altdeg=30)
hillshade = ls.hillshade(bathy_z, vert_exag=1)

valid_bathy_z = bathy_z[~np.isnan(bathy_z)]
norm = Normalize(vmin=np.nanmin(valid_bathy_z)+100, vmax=np.nanmax(valid_bathy_z)+400)

masked_bathy_z = np.ma.array(bathy_z, mask=np.isnan(bathy_z))


cmap = plt.cm.YlGnBu_r
rgba = cmap(norm(masked_bathy_z))

shaded = ls.shade(masked_bathy_z, cmap=cmap, norm=norm, blend_mode='overlay', vert_exag=0.5, dx=1, dy=1, fraction=0.7)

ax1.imshow(shaded, extent=(bathy_x.min(), bathy_x.max(), bathy_y.min(), bathy_y.max()),
          transform=ccrs.PlateCarree(), zorder=1)

ax1 = plt.gca()
ax1.set_xlim(xlim)
ax1.set_ylim(ylim);

#ax1.text(-129.04, 48.021, 'Endeavour Segment', transform=ccrs.PlateCarree(), ha='center', color='black', size=20,\
#           path_effects=[pe.withStroke(linewidth=0, foreground="black")],zorder=2,\
#           bbox=dict(facecolor='white', edgecolor='none', alpha=0.7, boxstyle='round,pad=0.1')) 

bpr_lons = [mothra_bpr_lon, mef_bpr_lon, south_bpr_lon, north_bpr_lon, east_bpr_lon]
bpr_lats = [mothra_bpr_lat,  mef_bpr_lat, south_bpr_lat, north_bpr_lat, east_bpr_lat]
bpr_labels = ['Mothra',  'MEF', 'South', 'North', 'East']
label_colors = ['C2', 'C4', 'C3', 'C1', 'C0']
bpr_names = ['Mothra','MEF', 'South', 'North', 'East']
colors = ['C2','C4', 'C3', 'C1', 'C0']


#bpr_lons = [mef_bpr_lon, south_bpr_lon, north_bpr_lon, east_bpr_lon]
#bpr_lats = [mef_bpr_lat, south_bpr_lat, north_bpr_lat, east_bpr_lat]
#colors = ['C0', 'C1', 'C2', 'C3']

for name, lon, lat,color in zip(bpr_labels, bpr_lons, bpr_lats, label_colors):
    ax1.scatter(lon, lat, s=300, color=color, marker='.', transform=ccrs.PlateCarree(), zorder=3,\
          path_effects=[pe.withStroke(linewidth=2, foreground="black")])
    ax1.text(lon+0.008, lat - 0.01, name, transform=ccrs.PlateCarree(), ha='center', color=color, size=15,\
           path_effects=[pe.withStroke(linewidth=1.5, foreground="black")],zorder=4,\
           bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.1'), 
) 
    
# Approximate coordinates of the five main vent fields (°N, °W)
vent_lats  = [47.923, 47.950, 47.967, 47.982, 47.997]          # Mothra, MEF, High Rise, Salty Dawg, Sasquatch
vent_lons  = [-129.109, -129.100, -129.090, -129.076, -129.066]
vent_names = ['Mothra', 'MEF', 'High Rise', 'Salty Dawg', 'Sasquatch']
vent_colors = ['C2', 'C4', 'C3', 'C1', 'C0']

for name, lon, lat, color in zip(vent_names, vent_lons, vent_lats, vent_colors):
    ax1.scatter(lon, lat, s=30, color='gold', marker='^', transform=ccrs.PlateCarree(),
                zorder=3, path_effects=[pe.withStroke(linewidth=2, foreground="black")])
    ax1.text(lon - 0.01, lat + 0.002, name, transform=ccrs.PlateCarree(),
             ha='center', color='k', size=12,\
           path_effects=[pe.withStroke(linewidth=1.5, foreground="w")],zorder=4,
             )

#se_lon, se_lat = (-129.0988930000000 + 360), 47.933395000000000
#ax1.scatter(se_lon, se_lat, s=10, color='C5', marker='.',
#            transform=ccrs.PlateCarree(), zorder=3,
#            path_effects=[pe.withStroke(linewidth=2, foreground="black")])
#ax1.text(se_lon+0.025, se_lat-0.001, 'Southeast', transform=ccrs.PlateCarree(), ha='center',
#         color='C5', size=15, zorder=4,
#         path_effects=[pe.withStroke(linewidth=1.5, foreground="black")],
#         bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.1'))
#
#ne_lon, ne_lat = (-129.0823133333000 + 360), 47.973705000000000
#ax1.scatter(ne_lon, ne_lat, s=10, color='C6', marker='.',
#            transform=ccrs.PlateCarree(), zorder=3,
#            path_effects=[pe.withStroke(linewidth=2, foreground="black")])
#ax1.text(ne_lon+0.025, ne_lat-0.001, 'Northeast', transform=ccrs.PlateCarree(), ha='center',
#         color='C6', size=15, zorder=4,
#         path_effects=[pe.withStroke(linewidth=1.5, foreground="black")],
#         bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.1'))
#

#glory_lon, glory_lat = (-129.0833 + 360), 47.9995
#ax1.scatter(glory_lon, glory_lat, s=300, color='C5', marker='.',
#            transform=ccrs.PlateCarree(), zorder=3,
#            path_effects=[pe.withStroke(linewidth=2, foreground="black")])
#ax1.text(glory_lon+0.004, glory_lat+0.006, 'GLORYS', transform=ccrs.PlateCarree(), ha='center',
#         color='C5', size=15, zorder=4,
#         path_effects=[pe.withStroke(linewidth=1.5, foreground="black")],
#         bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.1'))
#ctd_lon = -129.0354
#ctd_lat = 47.9584
#ax1.scatter(ctd_lon+0.003, ctd_lat, s=200, color='yellow', marker='*', transform=ccrs.PlateCarree(), zorder=3,\
          #path_effects=[pe.withStroke(linewidth=2, foreground="black")])
#ax1.text(ctd_lon+0.008, ctd_lat - 0.016, ' CTD', transform=ccrs.PlateCarree(), ha='center', color='yellow', size=15,\
           #path_effects=[pe.withStroke(linewidth=1.5, foreground="black")],zorder=2,\
           #bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.1')) 


#hycom_lon=230.8800048828125
#hycom_lat=47.959999084472656
#ax1.scatter(hycom_lon, hycom_lat, s=200, color='purple', marker='.', transform=ccrs.PlateCarree(), zorder=3,\
#         path_effects=[pe.withStroke(linewidth=2, foreground="black")])
#ax1.text(hycom_lon+0.004, hycom_lat + 0.004, 'HYCOM', transform=ccrs.PlateCarree(), ha='center', color='purple', size=15,\
#           path_effects=[pe.withStroke(linewidth=1.5, foreground="black")],zorder=2,\
#           bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.1')) 

xticks = np.linspace(xlim[0]+0.02, xlim[1]-0.015,5)
ax1.set_xticks(xticks, crs = ccrs.PlateCarree())
lon_formatter = LongitudeFormatter(zero_direction_label=True)
ax1.xaxis.set_major_formatter(lon_formatter)

ax1.set_yticks(np.linspace(ylim[0]+0.03, ylim[1]-0.01, 3), crs = ccrs.PlateCarree())
lat_formatter = LatitudeFormatter()
ax1.yaxis.set_major_formatter(lat_formatter)
plt.yticks(rotation=0);

add_scalebar(ax1, center_lon=-129.02, center_lat=47.90, length_km=2)


sm = ScalarMappable(cmap=cmap, norm=norm)
sm.set_array([])  
sm.set_clim(np.nanmin(valid_bathy_z)+100, np.nanmax(valid_bathy_z)+400)

#cb = plt.colorbar(sm, ax=ax1, shrink=0.5, aspect=15) 
#cb.set_label('Depth (m)', rotation=270, labelpad=15) 
#pos = cb.ax.get_position()
#new_pos = [pos.x0, pos.y0, pos.width, pos.height]  
#cb.ax.set_position(new_pos)

hline=960
latitudes = bathy_y[hline, :]
longitudes = bathy_x[hline, :]

ax1.plot(longitudes, latitudes, transform=ccrs.PlateCarree(), color='black', linestyle='--',\
         linewidth=1.4, label='Cross-section at Index {}'.format(hline), zorder=2)

ax1.grid(True, color='black',alpha=0.4);

divider = make_axes_locatable(ax1)


#________________________________________________________________________________________________________

ax2 = divider.new_vertical(
        size="40%", pad=0.5, axes_class=plt.Axes, pack_start=True
    )  
fig.add_axes(ax2)
cross_section = bathy_z[hline, :]/1000
ax2.plot(bathy_x[hline,:], cross_section, label='Bathymetric profile', color='black',linestyle='-')
ax2.set_ylabel('Depth (km bsf)')

ax2.grid(True);
ax2.set_xlim(ax1.get_xlim())

ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(xticks)
ax2.xaxis.set_major_formatter(lon_formatter)
ax2.set_ylim(-2450/1000, -2000/1000)
#ax2.legend()

# -----------------------------
# Compute horizontal distances
# -----------------------------
# -----------------------------
# Compute horizontal distances
# -----------------------------
from pyproj import Geod
geod = Geod(ellps="WGS84")

profile_lons = bathy_x[hline, :]
profile_lats = bathy_y[hline, :]
depths = bathy_z[hline, :]  # in meters

# Get longitude limits from ax1
lon_min, lon_max = ax1.get_xlim()

# Filter valid points within longitude limits and finite depth
valid = (
    ~np.isnan(depths) &
    (profile_lons >= lon_min) & (profile_lons <= lon_max)
)
profile_lons = profile_lons[valid]
profile_lats = profile_lats[valid]
depths = depths[valid]

# Compute cumulative distances
dists = [0]
for i in range(1, len(profile_lons)):
    _, _, dist = geod.inv(profile_lons[i-1], profile_lats[i-1], profile_lons[i], profile_lats[i])
    dists.append(dists[-1] + dist)
dists = np.array(dists)  # in meters

# -----------------------------
# Calculate steepest gradient (in degrees)
# -----------------------------
local_slopes = np.diff(depths) / np.diff(dists)  # unit: m/m
steepest_idx = np.argmax(np.abs(local_slopes))
steepest_rad = np.arctan(local_slopes[steepest_idx])
steepest_deg = abs(np.degrees(steepest_rad))

# -----------------------------
# Annotate location of steepest slope
# -----------------------------
# Get midpoint of the steepest segment
mid_lon = (profile_lons[steepest_idx] + profile_lons[steepest_idx + 1]) / 2
mid_depth = (depths[steepest_idx] + depths[steepest_idx + 1]) / 2

# Mark the steepest point on the profile
ax2.plot(mid_lon, mid_depth / 1000, 'rv', markersize=8, label='Steepest point')

# -----------------------------
# Annotate on plot
# -----------------------------
ax2.text(0.6, 0.9, f"Steepest gradient: {steepest_deg:.2f}°", transform=ax2.transAxes,
         fontsize=12, bbox=dict(facecolor='white', alpha=0.7, boxstyle='round'))


# ----------------------------------
# Compute gradient between -129.109 and -129.015
# ----------------------------------

# Define the two segments
segments = [(-129.109, -129.015), (-129.109, -129.14)]

for seg_start, seg_end in segments:
    # Find nearest indices
    idx_start = np.argmin(np.abs(profile_lons - seg_start))
    idx_end = np.argmin(np.abs(profile_lons - seg_end))

    # Ensure correct order
    if idx_start > idx_end:
        idx_start, idx_end = idx_end, idx_start

    # Extract segment
    lon_segment = profile_lons[idx_start:idx_end + 1]
    lat_segment = profile_lats[idx_start:idx_end + 1]
    depth_segment = depths[idx_start:idx_end + 1]

    # Compute horizontal distance (m)
    dist_segment = 0
    for i in range(len(lon_segment) - 1):
        _, _, d = geod.inv(lon_segment[i], lat_segment[i], lon_segment[i + 1], lat_segment[i + 1])
        dist_segment += d

    # Compute vertical change
    depth_change = depth_segment[-1] - depth_segment[0]

    # Compute gradient
    slope_rad = np.arctan(depth_change / dist_segment)
    slope_deg = abs(np.degrees(slope_rad))

    # Plot segment as dashed line
    ax2.plot([lon_segment[0], lon_segment[-1]],
             [depth_segment[0] / 1000, depth_segment[-1] / 1000],
             'k--', label=f'Segment {seg_start} to {seg_end}')

    # Annotate gradient
    mid_lon = (lon_segment[0] + lon_segment[-1]) / 2
    mid_depth = (depth_segment[0] + depth_segment[-1]) / 2
    ax2.text(mid_lon + 0.01, mid_depth / 1000 - 0.2,
             f"gradient: {slope_deg:.2f}°", fontsize=12,
             color='black', ha='center', va='bottom')

plt.tight_layout()





#plt.savefig('Endeavor_BPR_map_cross_section_presentation.png', dpi=300, bbox_inches='tight')

## Oregon Map

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from matplotlib.colors import LightSource
import matplotlib.patheffects as pe
import pandas as pd
from matplotlib import gridspec

# --- Load bathymetry dataset ---
ds = xr.open_dataset(r"H:\Chapter3\axial_endeavour_map\oregon.nc")

# --- Extract metadata ---
x_min, x_max = ds['x_range'].values
y_min, y_max = ds['y_range'].values
dx, dy = ds['spacing'].values
nx, ny = ds['dimension'].values.astype(int)

# --- Reconstruct coordinates ---
lon = np.linspace(x_min, x_max, nx)
lat = np.linspace(y_min, y_max, ny)
z = ds['z'].values.reshape((ny, nx))
z[z > 0] = np.nan  # Mask land

# --- Hillshade effect ---
ls = LightSource(azdeg=290, altdeg=10)
z_masked = np.ma.array(z, mask=np.isnan(z))
shaded = ls.shade(z_masked, cmap=plt.cm.YlGnBu_r, blend_mode='overlay',
                  vert_exag=2, dx=1, dy=1, fraction=0.7)



# --- Create combined figure ---
fig = plt.figure(figsize=(8, 6))
gs = gridspec.GridSpec(2, 1, height_ratios=[5, 2], hspace=0.3)



# =======================
# === Top: Bathymetry ===
# =======================
ax = fig.add_subplot(gs[0, 0], projection=ccrs.PlateCarree())
ax.set_xlim([-126, -124])
ax.set_ylim([44, 45])
map_xlim = ax.get_xlim()

ax.imshow(shaded, extent=(x_min, x_max, y_min, y_max),
          transform=ccrs.PlateCarree(), zorder=1)

profile_file = r"H:\Chapter3\LCH\oregon_profile.xyz"

# Skip first 4 header lines and read only valid numeric rows
df_profile = pd.read_csv(
    profile_file,
    sep=r'\s+',
    skiprows=4,
    names=['Longitude', 'Latitude', 'Elevation', 'Cumulative_km', 'Distance_km'],
    engine='python'
)

# Drop any rows that still contain non-numeric values
df_profile = df_profile[pd.to_numeric(df_profile['Longitude'], errors='coerce').notnull()]

# Convert columns to float (now safe)
df_profile = df_profile.astype(float)


# Ensure longitudes are interpreted in the correct range
df_profile['Longitude'] = ((df_profile['Longitude'] + 180) % 360) - 180


df_profile = df_profile.dropna()

# --- Plot profile as a line on the map ---
ax.plot(df_profile['Longitude'], df_profile['Latitude'],
        transform=ccrs.PlateCarree(), linestyle='--',
        color='black', linewidth=2, zorder=1)

# --- Add Oregon label ---
ax.text(-124.3, 44.88, 'Oregon', transform=ccrs.PlateCarree(), ha='center',
        color='black', size=24,
        path_effects=[pe.withStroke(linewidth=1.5, foreground="white")],
        bbox=dict(facecolor='white', edgecolor='none', alpha=0.9, boxstyle='round,pad=0.1'),
        zorder=5)



# --- Gridlines ---
gl = ax.gridlines(draw_labels=True, alpha=0.3)
gl.top_labels = False
gl.right_labels = False



PN1A_lon = -125.3983
PN1A_lat = 44.5096
PN1B_lon = -125.1483
PN1B_lat = 44.4815
PN1C_lat = 44.3636
PN1C_lon = -124.962
PN1D_lat = 44.6914
PN1D_lon = -124.457

# --- Add scatter point ---
s = 300
ax.scatter(PN1A_lon, PN1A_lat, transform=ccrs.PlateCarree(), s=s, color='C0',
           marker='.', path_effects=[pe.withStroke(linewidth=2, foreground="black")])
ax.text(PN1A_lon+0.005, PN1A_lat - 0.08, 'PN1A', transform=ccrs.PlateCarree(), ha='center', color='C0', size=size,\
           path_effects=[pe.withStroke(linewidth=1.5, foreground="black")],zorder=2,\
           bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.1')) 
# --- Add PN1B ---
ax.scatter(PN1B_lon, PN1B_lat, transform=ccrs.PlateCarree(), s=s, color='C1',
           marker='.', path_effects=[pe.withStroke(linewidth=2, foreground="black")])
ax.text(PN1B_lon+0.005, PN1B_lat - 0.08, 'PN1B', transform=ccrs.PlateCarree(), ha='center', color='C1', size=size,
           path_effects=[pe.withStroke(linewidth=1.5, foreground="black")], zorder=2,
           bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.1'))

# --- Add PN1C ---
ax.scatter(PN1C_lon, PN1C_lat, transform=ccrs.PlateCarree(), s=s, color='C2',
           marker='.', path_effects=[pe.withStroke(linewidth=2, foreground="black")])
ax.text(PN1C_lon+0.005, PN1C_lat - 0.08, 'PN1C', transform=ccrs.PlateCarree(), ha='center', color='C2', size=size,
           path_effects=[pe.withStroke(linewidth=1.5, foreground="black")], zorder=2,
           bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.1'))

# --- Add PN1D ---
ax.scatter(PN1D_lon, PN1D_lat, transform=ccrs.PlateCarree(), s=s, color='C3',
           marker='.', path_effects=[pe.withStroke(linewidth=2, foreground="black")])
ax.text(PN1D_lon+0.005, PN1D_lat - 0.08, 'PN1D', transform=ccrs.PlateCarree(), ha='center', color='C3', size=size,
           path_effects=[pe.withStroke(linewidth=1.5, foreground="black")], zorder=2,
           bbox=dict(facecolor='white', edgecolor='none', alpha=0.6, boxstyle='round,pad=0.1'))

add_scalebar(ax, center_lon=-125.9, center_lat=44.1, length_km=20)
# After drawing the map:
lon_min, lon_max = ax.get_xlim()






# ========================
# === Bottom: Profile ===
# ========================
ax2 = fig.add_subplot(gs[1, 0])
ax2.plot(df_profile['Longitude'], df_profile['Elevation']/1000, color='black', linestyle='-')
ax2.set_xlim(map_xlim)
ax2.set_ylabel('Depth (km bsf)')
ax2.grid(True)



# --- Compute steepest slope using df_profile ---
lons = df_profile['Longitude'].values
elevs = df_profile['Elevation'].values

valid = ~np.isnan(elevs) & ~np.isnan(lons)
lons = lons[valid]
elevs = elevs[valid]

# Approximate horizontal distance using degrees -> meters (1 deg lon ≈ 111 km * cos(lat))
# Use mean latitude for scaling
mean_lat = np.nanmean(df_profile['Latitude'].values)
deg2m = 111e3 * np.cos(np.radians(mean_lat))
dx = np.diff(lons) * deg2m  # in meters
dz = np.diff(elevs)

local_slopes = dz / dx  # slope = rise/run
steepest_idx = np.argmax(np.abs(local_slopes))
steepest_slope_deg = np.degrees(np.arctan(local_slopes[steepest_idx]))

# Midpoint coordinates for annotation
mid_lon = (lons[steepest_idx] + lons[steepest_idx + 1]) / 2
mid_elev = (elevs[steepest_idx] + elevs[steepest_idx + 1]) / 2

# --- Plot the annotation and marker ---
ax2.text(0.05, 0.95, f"Steepest gradient: {steepest_slope_deg:.2f}°", transform=ax2.transAxes,
         fontsize=14, bbox=dict(facecolor='white', alpha=0.7, boxstyle='round'))

# ------two gradient lines

# Define target longitudes
lon1, lon2 = -124.5, -125.1
lon3, lon4 = -125.2, -125.35  # Change these to your desired second segment

for lon_start, lon_end in [(lon1, lon2), (lon3, lon4)]:
    # Find nearest points in the profile data
    idx_start = (np.abs(df_profile['Longitude'] - lon_start)).idxmin()
    idx_end = (np.abs(df_profile['Longitude'] - lon_end)).idxmin()

    # Extract relevant data
    lon_a = df_profile.loc[idx_start, 'Longitude']
    elev_a = df_profile.loc[idx_start, 'Elevation']
    dist_a = df_profile.loc[idx_start, 'Cumulative_km']

    lon_b = df_profile.loc[idx_end, 'Longitude']
    elev_b = df_profile.loc[idx_end, 'Elevation']
    dist_b = df_profile.loc[idx_end, 'Cumulative_km']

    # Ensure correct ordering
    if dist_a > dist_b:
        lon_a, lon_b = lon_b, lon_a
        elev_a, elev_b = elev_b, elev_a
        dist_a, dist_b = dist_b, dist_a

    # Calculate horizontal distance and slope
    dx_km = abs(dist_b - dist_a)  # in kilometers
    dz_m = abs(elev_b - elev_a)   # in meters

    slope_deg = np.degrees(np.arctan(dz_m / (dx_km * 1000)))  # convert km to meters

    # Plot the straight line segment
    ax2.plot([lon_a, lon_b], [elev_a / 1000, elev_b / 1000], 'k--', label='Gradient segment')

    # Annotate the gradient
    mid_lon = (lon_a + lon_b) / 2
    mid_elev = (elev_a + elev_b) / 2
    ax2.text(mid_lon+0.3, mid_elev / 1000 + 0.05,
             f"gradient: {slope_deg:.2f}°", fontsize=12, ha='center', color='black')


# Marker on steepest point
ax2.plot(mid_lon, mid_elev / 1000, 'rv', markersize=8, label='Steepest point')  # 'rv' = red triangle
#ax2.legend()



# Use the same ticks for both top and bottom explicitly
# --- Force tick alignment with top map ---
xticks = np.array([ -125.5, -125.0, -124.5])
ax2.set_xticks(xticks)
ax2.set_xticklabels([f"{abs(t):.2f}°W" for t in xticks],fontsize=10)



plt.tight_layout()
plt.show()

